# CNN Feature Extraction

This script runs any CNN model to: 
* Extract feature, and put them to an output csv.
* For each test image, find the best matching one, get its coordinates and output.

In [1]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from matplotlib import pyplot as plt
import pickle

import tensorflow as tf
from keras.applications.resnet import *
from keras.preprocessing.image_dataset import *
from keras.preprocessing import image
from keras.models import Model

## Reading in data

In [2]:
train_i = []
test_i = []

train_xy = pd.read_csv('train.csv')
train_path = train_xy['id'].values
test_path = pd.read_csv('imagenames.csv')['id'].values

for f in train_path:
    img = image.load_img('./train/' + f + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    train_i.append(x)
for f in test_path:
    img = image.load_img('./test/' + f + '.jpg', target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    test_i.append(x)

## Setting up CNN model

In [3]:
# The chosen model is ResNet101
# Other available models: ResNet50, ResNet152, ResNet..V2, VGG16, VGG19
base_model = ResNet101(weights = 'imagenet', include_top = True)
# Get the final FC layer. It contains minimal information, but enough for usability
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
model.summary()

179527680/179518384 [==============================] - 31s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
__________________________

## Train and export the extracted features to Pickle files

In [4]:
train_preds = []
test_preds = []

for i in tqdm(train_i):
    pred = model.predict(i)
    train_preds.append(pred)

for i in tqdm(test_i):
    pred = model.predict(i)
    test_preds.append(pred)

f = open(f'./test_resnet101.pckl','wb')
pickle.dump(test_preds,f)
f.close()

f = open(f'./train_resnet101.pckl','wb')
pickle.dump(train_preds,f)
f.close()

100%|██████████| 1200/1200 [03:04<00:00,  6.49it/s]


## Also, export the feature similarity of training images with respect to testing images
* The chosen measurement is Euclidean distance

In [6]:
best_train = []
for test in tqdm(test_preds):
    match = [np.sum((train-test)**2)**0.5 for train in train_preds]
    best_match = np.argsort(match)#[:3]
    row = train_xy.iloc[best_match]['id'].values
    best_train.append(row)

df = pd.DataFrame(best_train, columns=[f'{i+1}' for i in range(7500)])
df.to_csv('resnet101_matches.csv',index=False)

100%|██████████| 1200/1200 [01:45<00:00, 11.35it/s]


## Export the average of top 3 best matching image for each test image

In [7]:
out=[]
for i in range(len(df)):
    coor = [train_xy[train_xy['id']== df[label][i]] for label in ['1','2','3']]
    result = pd.concat(coor)
    out.append([test_path[i], np.mean(result['x']), np.mean(result['y'])])
    
out_csv = pd.DataFrame(out, columns=['id','x','y'])
out_csv.to_csv('resnet101v2_top3_out.csv',index=False)